In [10]:
import os
from google.cloud import bigquery
from get_bq_data import get_bq_data
from model_trainer_1 import get_predictions
from dotenv import load_dotenv

load_dotenv()

os.environ['GOOGLE_APPLICATION_CREDENTIALS' ] = 'service_keys.json'

client = bigquery. Client()

In [11]:
def aggregate_data():
    """Load dataset, aggregate, and push to Supabase""" 

    existing_data = get_bq_data(client,"state_market") 

    return existing_data

In [12]:
# def train_model(df):
#     """Load dataset, train model, save predictions to Supabase"""

#     # List of features to predict
#     features = [
#         "median_listing_price",
#         "average_listing_price",
#         "median_listing_price_per_square_foot",
#         "total_listing_count",
#         "median_days_on_market"
#     ]
    
#     for feature in features:
#         get_predictions(df, feature)

In [13]:
df = aggregate_data()
df.sort_values(by=['year', 'month', 'state_num'], inplace=True)
df.reset_index(drop=True, inplace=True)
df

,year,month,state_num,median_listing_price,median_listing_price_mm,median_listing_price_yy,active_listing_count,active_listing_count_mm,active_listing_count_yy,median_days_on_market,...,median_square_feet_yy,average_listing_price,average_listing_price_mm,average_listing_price_yy,total_listing_count,total_listing_count_mm,total_listing_count_yy,pending_ratio,pending_ratio_mm,pending_ratio_yy
0,2017,7,1,198060,-0.004223,0.0654,26519,0.002343,-0.1296,75,...,0.0233,264342,-0.000707,0.0543,28563,-0.004288,-0.1169,0.0789,-0.067376,0.0179
1,2017,7,2,279375,-0.006667,-0.0333,3710,0.092784,-0.0196,53,...,-0.0278,327328,-0.003374,-0.0121,3710,0.092784,-0.0196,0.0000,0.000000,0.0000
2,2017,7,3,312000,-0.018868,0.0576,26807,-0.026015,-0.1148,56,...,0.0086,477781,-0.029825,0.0699,34886,-0.034965,-0.0948,0.2988,-0.048711,0.0294
3,2017,7,4,169900,0.000000,0.0534,15717,-0.013309,-0.1057,73,...,0.0245,235007,0.003810,0.0652,18446,-0.013161,-0.0939,0.1771,0.002831,0.0145
4,2017,7,5,549000,-0.001637,0.0660,71686,0.037319,-0.1431,40,...,-0.0005,1030673,-0.016759,0.1016,106530,0.009907,-0.1262,0.4944,-0.069627,0.0314
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5044,2025,9,47,450000,0.000000,0.0228,21543,0.028400,0.2931,45,...,0.0005,639805,0.012100,0.0062,31820,0.022700,0.2011,0.4810,-0.011600,-0.1128
5045,2025,9,48,648900,-0.001600,-0.0016,24463,-0.002500,0.2562,57,...,-0.0035,887686,-0.012300,-0.0109,33936,-0.010900,0.1543,0.3888,-0.011200,-0.1228
5046,2025,9,49,261500,-0.027900,0.0898,4245,0.038700,0.1546,52,...,0.0120,325854,-0.029700,0.0233,6712,0.030100,0.1029,0.5822,-0.010600,-0.0729
5047,2025,9,50,392325,-0.016700,0.0257,13190,0.037400,0.0899,44,...,-0.0036,506718,-0.004000,0.0422,18863,0.000800,0.0766,0.4327,-0.049800,-0.0216


In [14]:
from preprocessing_2 import preprocess_data_2

target_df = preprocess_data_2(25, df.copy())
prediction_df = target_df.copy()

In [15]:
def train_model(df, prediction_df):
    """Load dataset, train model, save predictions to Supabase"""

    features = [
        "median_listing_price",
        "median_days_on_market"
    ]
    
    for feature in features:
        predictions = get_predictions(df, feature, 24)
        prediction_df[feature] = predictions

    for col in features:
        if col in prediction_df.columns:
            prediction_df[col] = prediction_df[col].astype(int)

    return prediction_df

In [16]:
preds = train_model(df, prediction_df)

Initial memory: 1.80 MB
Final memory: 1.20 MB
Reduced by 33.2%
Validation RMSE: 25758.3877, RMSLE: 0.0576, MAE: 20027.8076, MAPE: 4.53%, R²: 0.9684
Validation RMSE: 24591.6228, RMSLE: 0.0579, MAE: 19875.8159, MAPE: 4.63%, R²: 0.9707
Validation RMSE: 24912.9814, RMSLE: 0.0585, MAE: 20113.1023, MAPE: 4.70%, R²: 0.9691
Validation RMSE: 23030.3157, RMSLE: 0.0582, MAE: 19681.4635, MAPE: 4.81%, R²: 0.9735
Validation RMSE: 21073.4184, RMSLE: 0.0485, MAE: 16258.8128, MAPE: 3.77%, R²: 0.9783
Validation RMSE: 37272.8521, RMSLE: 0.0910, MAE: 31277.6786, MAPE: 7.34%, R²: 0.9331
Validation RMSE: 48528.4525, RMSLE: 0.1063, MAE: 39186.1468, MAPE: 8.77%, R²: 0.8870
Validation RMSE: 60313.0797, RMSLE: 0.1443, MAE: 53695.0836, MAPE: 12.08%, R²: 0.8332
Validation RMSE: 77465.1050, RMSLE: 0.1804, MAE: 69075.0726, MAPE: 15.23%, R²: 0.7142
Validation RMSE: 91817.1577, RMSLE: 0.2163, MAE: 81119.1544, MAPE: 17.78%, R²: 0.5991
Validation RMSE: 59468.9872, RMSLE: 0.1317, MAE: 51184.2574, MAPE: 11.10%, R²: 0.824

In [21]:
preds2 = preds.copy()

In [62]:
latest_df = df.groupby("state_num").tail(1).reset_index(drop=True)
preds['year_month'] = (preds['year'] - 2025) * 12 + preds['month'] - 9
preds.drop(columns=['year', 'month'], inplace=True)
cols_order = ['year_month', 'state_num', 'median_listing_price', 'median_days_on_market']
preds = preds[cols_order]

In [63]:
preds = preds[preds['year_month'].isin([3,6,12])]
preds.loc[preds['year_month'] == 12, 'median_listing_price'] = preds.loc[preds['year_month'] == 12, 'median_listing_price'] * 1.15
preds.loc[preds['year_month'] == 6, 'median_listing_price'] = preds.loc[preds['year_month'] == 6, 'median_listing_price'] * 1.1
preds

,year_month,state_num,median_listing_price,median_days_on_market
102,3,1,309099.0,79
103,3,2,410331.0,99
104,3,3,483349.0,70
105,3,4,278332.0,77
106,3,5,721191.0,69
...,...,...,...,...
607,12,47,504757.2,48
608,12,48,705543.6,47
609,12,49,286576.8,62
610,12,50,400951.2,48


In [64]:
preds['appreciation'] = 0.0
for index, row in preds.iterrows():
    state_num = row['state_num']
    
    current_price = latest_df[(latest_df['state_num'] == state_num)]['median_listing_price'].values[0]
    predicted_price = row['median_listing_price']
    
    appreciation = ((predicted_price - current_price) / current_price) * 100
    preds.loc[index, 'appreciation'] = appreciation

In [72]:
import numpy as np

def calculate_volatility(df):
    volatility = {}
    for state in df['state_num'].unique():
        state_data = df[df['state_num'] == state].sort_values(by=['year', 'month'])
        if len(state_data) >= 12:
            last_12_mm = state_data['median_listing_price_mm'].tail(12).values
            volatility[state] = np.std(last_12_mm) * 100
        else:
            volatility[state] = np.nan  # Not enough data to calculate volatility
    return volatility

volatility_data = calculate_volatility(df)

preds['volatility'] = 0.0  # Initialize Volatility column

for index, row in preds.iterrows():
    state_num = row['state_num']
    if state_num in volatility_data:
        preds.loc[index, 'volatility'] = volatility_data[state_num]
    else:
        preds.loc[index, 'volatility'] = np.nan

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

def normalize_days_on_market(preds):
    # Use transform so the inverse values align with the original index
    inv_days = preds.groupby('year_month')['median_days_on_market'].transform(
        lambda s: np.where(s != 0, 1 / s, 0)
    )

    scaler = MinMaxScaler()
    preds = preds.copy()
    preds['liquidity'] = scaler.fit_transform(inv_days.to_frame()) * 100
    return preds

preds = normalize_days_on_market(preds)
preds

TypeError: incompatible index of inserted column with frame index